# Evaluating Prompt Effectiveness

## Overview
This tutorial focuses on methods and techniques for evaluating the effectiveness of prompts in AI language models. We'll explore various metrics for measuring prompt performance and discuss both manual and automated evaluation techniques.

## Motivation
As prompt engineering becomes increasingly crucial in AI applications, it's essential to have robust methods for assessing prompt effectiveness. This enables developers and researchers to optimize their prompts, leading to better AI model performance and more reliable outputs.

## Key Components
1. Metrics for measuring prompt performance
2. Manual evaluation techniques
3. Automated evaluation techniques
4. Practical examples using OpenAI and LangChain

## Method Details
We'll start by setting up our environment and introducing key metrics for evaluating prompts. We'll then explore manual evaluation techniques, including human assessment and comparative analysis. Next, we'll delve into automated evaluation methods, utilizing techniques like perplexity scoring and automated semantic similarity comparisons. Throughout the tutorial, we'll provide practical examples using OpenAI's GPT models and LangChain library to demonstrate these concepts in action.

## Conclusion
By the end of this tutorial, you'll have a comprehensive understanding of how to evaluate prompt effectiveness using both manual and automated techniques. You'll be equipped with practical tools and methods to optimize your prompts, leading to more efficient and accurate AI model interactions.

## Setup

First, let's import the necessary libraries and set up our environment.

In [ ]:
import os
from langchain_openai import ChatOpenAI
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini")

# Initialize sentence transformer for semantic similarity
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

def semantic_similarity(text1, text2):
    """Calculate semantic similarity between two texts using cosine similarity."""
    embeddings = sentence_model.encode([text1, text2])
    return cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]

## Metrics for Measuring Prompt Performance

Let's define some key metrics for evaluating prompt effectiveness:

In [8]:
def relevance_score(response, expected_content):
    """Calculate relevance score based on semantic similarity to expected content."""
    return semantic_similarity(response, expected_content)

def consistency_score(responses):
    """Calculate consistency score based on similarity between multiple responses."""
    if len(responses) < 2:
        return 1.0  # Perfect consistency if there's only one response
    similarities = []
    for i in range(len(responses)):
        for j in range(i+1, len(responses)):
            similarities.append(semantic_similarity(responses[i], responses[j]))
    return np.mean(similarities)

def specificity_score(response):
    """Calculate specificity score based on response length and unique word count."""
    words = response.split()
    unique_words = set(words)
    return len(unique_words) / len(words) if words else 0

## Manual Evaluation Techniques

Manual evaluation involves human assessment of prompt-response pairs. Let's create a function to simulate this process:

In [4]:
def manual_evaluation(prompt, response, criteria):
    """Simulate manual evaluation of a prompt-response pair."""
    print(f"Prompt: {prompt}")
    print(f"Response: {response}")
    print("\nEvaluation Criteria:")
    for criterion in criteria:
        score = float(input(f"Score for {criterion} (0-10): "))
        print(f"{criterion}: {score}/10")
    print("\nAdditional Comments:")
    comments = input("Enter any additional comments: ")
    print(f"Comments: {comments}")

# Example usage
prompt = "Explain the concept of machine learning in simple terms."
response = llm.invoke(prompt).content
criteria = ["Clarity", "Accuracy", "Simplicity"]
manual_evaluation(prompt, response, criteria)

Prompt: Explain the concept of machine learning in simple terms.
Response: Machine learning is a type of computer technology that allows computers to learn from data and improve their performance over time without being explicitly programmed for every specific task. 

In simple terms, imagine teaching a child to recognize different animals. Instead of giving them a detailed description of each animal, you show them many pictures of cats, dogs, and birds. Over time, the child learns to identify these animals based on patterns they see in the images, like shapes, colors, and sizes. 

In the same way, machine learning involves feeding a computer lots of data (like pictures, numbers, or text) and letting it figure out patterns and make decisions on its own. For example, a machine learning model can be trained to recognize spam emails by analyzing examples of both spam and non-spam messages. Once trained, it can then automatically identify new emails as spam or not.

So, in essence, machine

## Automated Evaluation Techniques

Now, let's implement some automated evaluation techniques:

In [9]:
def automated_evaluation(prompt, response, expected_content):
    """Perform automated evaluation of a prompt-response pair."""
    relevance = relevance_score(response, expected_content)
    specificity = specificity_score(response)
    
    print(f"Prompt: {prompt}")
    print(f"Response: {response}")
    print(f"\nRelevance Score: {relevance:.2f}")
    print(f"Specificity Score: {specificity:.2f}")
    
    return {"relevance": relevance, "specificity": specificity}

# Example usage
prompt = "What are the three main types of machine learning?"
expected_content = "The three main types of machine learning are supervised learning, unsupervised learning, and reinforcement learning."
response = llm.invoke(prompt).content
automated_evaluation(prompt, response, expected_content)

Prompt: What are the three main types of machine learning?
Response: The three main types of machine learning are:

1. **Supervised Learning**: In supervised learning, the model is trained on a labeled dataset, which means that the input data is paired with the correct output. The goal is for the model to learn to map inputs to the correct outputs so that it can make predictions on new, unseen data. Common applications include classification (e.g., spam detection) and regression (e.g., predicting house prices).

2. **Unsupervised Learning**: In unsupervised learning, the model is trained on data that does not have labeled outputs. The goal is to identify patterns, structures, or relationships within the data. Common techniques include clustering (e.g., grouping customers based on purchasing behavior) and dimensionality reduction (e.g., reducing the number of features while retaining important information).

3. **Reinforcement Learning**: In reinforcement learning, an agent learns to ma

{'relevance': 0.73795843, 'specificity': 0.6403940886699507}

## Comparative Analysis

Let's compare the effectiveness of different prompts for the same task:

In [6]:
def compare_prompts(prompts, expected_content):
    """Compare the effectiveness of multiple prompts for the same task."""
    results = []
    for prompt in prompts:
        response = llm.invoke(prompt).content
        evaluation = automated_evaluation(prompt, response, expected_content)
        results.append({"prompt": prompt, **evaluation})
    
    # Sort results by relevance score
    sorted_results = sorted(results, key=lambda x: x['relevance'], reverse=True)
    
    print("Prompt Comparison Results:")
    for i, result in enumerate(sorted_results, 1):
        print(f"\n{i}. Prompt: {result['prompt']}")
        print(f"   Relevance: {result['relevance']:.2f}")
        print(f"   Specificity: {result['specificity']:.2f}")
    
    return sorted_results

# Example usage
prompts = [
    "List the types of machine learning.",
    "What are the main categories of machine learning algorithms?",
    "Explain the different approaches to machine learning."
]
expected_content = "The main types of machine learning are supervised learning, unsupervised learning, and reinforcement learning."
compare_prompts(prompts, expected_content)

Prompt: List the types of machine learning.
Response: Machine learning can be broadly categorized into several types, each serving different purposes and applications. The main types of machine learning are:

1. **Supervised Learning**:
   - Involves training a model on a labeled dataset, where the input data is paired with the correct output. The model learns to map inputs to outputs, and its performance is evaluated based on how accurately it predicts the outcomes for new, unseen data.
   - Common algorithms: Linear regression, logistic regression, decision trees, support vector machines, neural networks.

2. **Unsupervised Learning**:
   - Involves training a model on data without labeled responses. The model tries to learn the underlying structure or distribution in the data, often identifying patterns, clusters, or relationships.
   - Common algorithms: K-means clustering, hierarchical clustering, principal component analysis (PCA), t-distributed stochastic neighbor embedding (t-S

[{'prompt': 'List the types of machine learning.',
  'relevance': 0.73586243,
  'specificity': 0.5693430656934306},
 {'prompt': 'Explain the different approaches to machine learning.',
  'relevance': 0.68791693,
  'specificity': 0.5223880597014925},
 {'prompt': 'What are the main categories of machine learning algorithms?',
  'relevance': 0.67862606,
  'specificity': 0.6039603960396039}]

## Putting It All Together

Now, let's create a comprehensive prompt evaluation function that combines both manual and automated techniques:

In [7]:
def evaluate_prompt(prompt, expected_content, manual_criteria=['Clarity', 'Accuracy', 'Relevance']):
    """Perform a comprehensive evaluation of a prompt using both manual and automated techniques."""
    response = llm.invoke(prompt).content
    
    print("Automated Evaluation:")
    auto_results = automated_evaluation(prompt, response, expected_content)
    
    print("\nManual Evaluation:")
    manual_evaluation(prompt, response, manual_criteria)
    
    return {"prompt": prompt, "response": response, **auto_results}

# Example usage
prompt = "Explain the concept of overfitting in machine learning."
expected_content = "Overfitting occurs when a model learns the training data too well, including its noise and fluctuations, leading to poor generalization on new, unseen data."
evaluate_prompt(prompt, expected_content)

Automated Evaluation:
Prompt: Explain the concept of overfitting in machine learning.
Response: Overfitting is a common problem in machine learning where a model learns not only the underlying patterns in the training data but also the noise and random fluctuations. This leads to a model that performs exceptionally well on the training dataset but poorly on unseen data or the test dataset. In essence, the model becomes overly complex, capturing details that do not generalize to new data points.

### Key Aspects of Overfitting:

1. **Complexity of the Model**: Overfitting often occurs when a model is too complex relative to the amount of training data available. For example, a high-degree polynomial regression may fit a small set of data points perfectly but will not generalize well to new data.

2. **Training vs. Validation Performance**: A clear sign of overfitting is when the performance metrics (such as accuracy, loss, etc.) on the training data are significantly better than those o

{'prompt': 'Explain the concept of overfitting in machine learning.',
 'response': "Overfitting is a common problem in machine learning where a model learns not only the underlying patterns in the training data but also the noise and random fluctuations. This leads to a model that performs exceptionally well on the training dataset but poorly on unseen data or the test dataset. In essence, the model becomes overly complex, capturing details that do not generalize to new data points.\n\n### Key Aspects of Overfitting:\n\n1. **Complexity of the Model**: Overfitting often occurs when a model is too complex relative to the amount of training data available. For example, a high-degree polynomial regression may fit a small set of data points perfectly but will not generalize well to new data.\n\n2. **Training vs. Validation Performance**: A clear sign of overfitting is when the performance metrics (such as accuracy, loss, etc.) on the training data are significantly better than those on the 